# REGULAR EXPRESSIONS

## Intro

Регулярные выражения - выражения для поиска и замены части текста в строке или файле. Для работы с ними необходимо подключить модуль **"re"** из стандартной библиотеки Python.

In [22]:
import re

Наиболее часто регулярные выражения используются для поиска в строке, разбиении строк, замены части строк. 

Also called "regex" or "regexp". It is smart "find" or "search" in the text.

The language comes form 1960's. It's **character** based.

[Documentation](https://docs.python.org/3/library/re.html)

This module provides regular expression matching operations similar to those found in Perl.

Both patterns and strings to be searched can be Unicode strings (str) as well as 8-bit strings (bytes). However, Unicode strings and 8-bit strings cannot be mixed: that is, you cannot match a Unicode string with a byte pattern or vice-versa; similarly, when asking for a substitution, the replacement string must be of the same type as both the pattern and the search string.

## Escape sequences `\`

Regular expressions use the backslash character `\` to indicate special forms or to allow special characters to be used without invoking their special meaning. 

> The backslash (\) character is used to escape characters that otherwise have a special meaning, such as newline, backslash itself, or the quote character. [source](https://docs.python.org/3/reference/lexical_analysis.html)

This collides with Python’s usage of the same character for the same purpose in string literals; for example, to match a literal backslash, one might have to write `\\\\` as the pattern string, because the regular expression must be `\\`, and each backslash must be expressed as `\\` (to let Python know we we want just one `\`) inside a regular Python string literal. 

In [13]:
a = '\'
a

SyntaxError: unterminated string literal (detected at line 1) (3986968758.py, line 1)

In [15]:
a = '\\'
y = re.findall('\\\\', a)
y

['\\']

Also, please note that any invalid escape sequences (игнорирование спецсимвола) in Python’s usage of the backslash in string literals now generate a `DeprecationWarning` and in the future this will become a `SyntaxError`. This behaviour will happen even if it is a valid escape sequence for a regular expression.

### Raw String usage `r'string'`

The solution is to use Python’s raw string notation for regular expression patterns; backslashes are not handled in any special way in a string literal prefixed with `'r'`. So `r"\n"` is a two-character string containing `'\'` and `'n'`, while `"\n"` is a one-character string containing a newline. Usually patterns will be expressed in Python code using this raw string notation.

In [20]:
a = r'\n'
b = '\n'
print('a:', a)
print('b:', b)

a: \n
b: 



In [21]:
a = '\\'
y = re.findall(r'\\', a)
y

['\\']

## `re` methods

### **re.match**(шаблон, строка) 
- ищет заданный шаблон с самого начала строки.

In [2]:
import re
print(re.match(r'Hey', 'Hey Hey'))

print('Данные нашлись')

<re.Match object; span=(0, 3), match='Hey'>
Данные нашлись


In [3]:
import re
print(re.match('Hey', 'hey Hey'))

print('Данные не нашлись, т.к. строка отличается от шаблона с первого символа.')
print('Обратите внимание на синтаксис, перед шаблоном ставится латинская буква r')
print('Ну-ну, буква r не нужна')

None
Данные не нашлись, т.к. строка отличается от шаблона с первого символа.
Обратите внимание на синтаксис, перед шаблоном ставится латинская буква r
Ну-ну, буква r не нужна


### **re.search**(шаблон, строка) 
- ищет заданный шаблон по всей строке, возвращает результат при первом совпадении.

In [10]:
import re
print(re.search('Hey', 'hey Hey').group(0))     # Добавляем метод group(), чтобы вывести содержимое поиска

Hey


### **re.findall**(шаблон, строка)
- ищет заданный шаблон и возвращает все совпадения в виде **списка**.

In [4]:
import re
print(re.findall('Hey', 'hey Hey Hey Hey'))

['Hey', 'Hey', 'Hey']


### **re.split**(шаблон, строка) 
- разделяет строку по заданному шаблону

In [13]:
import re
a = 'hey Hey Hey Hey'
print(re.split('y', a)) 

['he', ' He', ' He', ' He', '']


### **re.sub**(шаблон, замена, строка) 
- находит шаблон в строке и производит замену

In [14]:
import re
a = 'hey Hey Hey Hey'
print(re.sub('Hey', '?', a))

hey ? ? ?


### **re.compile**(шаблон) 
- позволяет собирать регулярное выражение в отдельный объект для последующего использования

In [15]:
ex_str = re.compile('Hey')

result = ex_str.findall('hey Hey Hey')
print (result)

result2 = ex_str.findall('Hey')
print (result2)

['Hey', 'Hey']
['Hey']


Перейдем к самой интересной части - представьте, что вам необходимо произвести чтение "грязного" файла и убрать из него все лишнее, но при этом вы не можете сказать точно, что необходимо убрать. Эту задачу можно решить, используя специальные символы:

### `.`   
Один любой символ, кроме символа переноса строки `\n`

In [10]:
print(re.findall(r'h..', 'hey Hey Hey Hey'))

['hey']
